In [11]:
import pandas as pd
import numpy as np
import glob
import sys
import pywt


# Fins all cnv files based on entity name
GLOB_PATH = "/home/alpha/programs/python_files/datasets/cnv_and_mut/"
ENTITY_NAME = ["gbm","k27","mng","oligo","pxa","astroLow","astroHigh","pa"]
FILE_TYPE = input("which file type to use? wes/igv \n")

if FILE_TYPE == "wes":
    POSTFIX = "_cnv/*.MAXDENS.segments.seg"
elif FILE_TYPE == "igv":
    POSTFIX = "_cnv/*.bins.igv"

# add all paths to a list
ENTITY_NAME_PATH = []
for entity in ENTITY_NAME:
    entityPath = glob.glob(GLOB_PATH + entity + POSTFIX)
    ENTITY_NAME_PATH.extend(entityPath)

# read the files into a dataframe
df_cnv = pd.DataFrame()
for entity in ENTITY_NAME_PATH:
    if FILE_TYPE == "seg":
        file = pd.read_csv(entity,sep="\t",header=0,names=["ID","chr","start","end","mark","bstat","pval","seg_mean","seg_median","alteration"])
        file["entity"] = entity.split("/")[-2]
        file["chr"] = file["chr"].replace(regex=r'[a-z]',value = '')
        file["chr"] = file["chr"].replace(regex=[r"X","Y"],value=[23,24])
        file["chr"] = file["chr"].astype("int")
    elif FILE_TYPE == "igv":
        file = pd.read_csv(entity,sep="\t",header=0,names=["chr","start","end","feature","value"])
        file = file[["feature",file.columns.tolist()[-1]]]
        file = file.set_index("feature")
        # file["value"] = 2*((file["value"]-file["value"].min())/(file["value"].max()-file["value"].min()))-1
        # file["value"] = (file["value"]-file["value"].mean())/file["value"].std()
        decomp_function = pd.Series(pywt.wavedec(file["value"], 'db1', level=3)[0])
        decomp_function = decomp_function.to_frame()
        decomp_function = decomp_function.T
        # extract the entity name from the path
        decomp_function["entity"] = entity.split("/")[-2] 
        # extract the ID from the path
        decomp_function["ID"] = entity.split("/")[-1].split(".")[0]
        # print(decomp_function.shape,decomp_function)
        # print(len(decomp_function))
        # 3883
        if decomp_function.shape == (1,1943):
            df_cnv = pd.concat([df_cnv,decomp_function])

df_cnv = df_cnv.set_index("ID")   
print(df_cnv.tail(2))

answer = input("do you what to save .cvs file? y/n \n")
if answer == "y":
    df_cnv.to_csv(GLOB_PATH + "entities_transformed.csv")
else:
    print("exiting program")

which file type to use? wes/igv 
 igv


                            0         1         2         3         4  \
ID                                                                      
205828600042_R08C01 -0.062225 -0.179605 -0.072832 -0.277539 -0.228395   
206600570023_R02C01 -0.015910 -0.120915 -0.050912 -0.215668 -0.035355   

                            5         6         7         8         9  ...  \
ID                                                                     ...   
205828600042_R08C01 -0.348957 -0.143543 -0.190212 -0.296278 -0.195869  ...   
206600570023_R02C01 -0.171827 -0.027577 -0.055508 -0.151321 -0.121269  ...   

                         1932      1933      1934      1935      1936  \
ID                                                                      
205828600042_R08C01 -0.267993 -0.109602 -0.096874 -0.078842 -0.209304   
206600570023_R02C01 -0.281075 -0.171473 -0.248548 -0.045962 -0.224506   

                         1937      1938      1939      1940  entity  
ID                             

do you what to save .cvs file? y/n 
 y


In [71]:
# plot cnv
fig = px.scatter(
    df2, x=x2, y=y2, color="chrom", color_continuous_scale="rdylgn"
)

fig.update_traces(marker_size=3)
fig.update_layout(width=1500, height=800)

fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['ID', 'chrom', 'loc.start', 'loc.end', 'num.mark', 'bstat', 'pval', 'seg.mean', 'seg.median', 'alteration'] but received: 84